# **Обработка данных**

In [ ]:
import math
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from skimage.io import imread
from datetime import datetime, timedelta
from zipfile import ZipFile
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

Находим нужную строку и столбец, содержащие информацию об эпицентре землетрясения

In [ ]:
def find_a_place(image, latitude, longitude):

    if latitude >= 0:
        string = math.floor(image.shape[0] / 2 * (90 - latitude) / 90)
    else:
        string = math.floor(-image.shape[0] / 2 * latitude / 90 + image.shape[0] / 2)

    if longitude >= 0:
        column = math.floor(image.shape[1] / 2 * (longitude) / 180 + image.shape[1] / 2)
    else:
        column = math.floor(image.shape[1] / 2 * (180 + longitude) / 180)

    return string, column


Находим зону землетрясения (это будет прямоугольник)

In [ ]:
def get_earthquake_zone(image, latitude, longitude, magnitude):

    length = math.floor(10 ** (0.43 * magnitude) / 111.1 * image.shape[0] / 180)
    width = math.floor(10 ** (0.43 * magnitude) / (111.1 * math.cos(latitude * math.pi / 180)) * (image.shape[1] / 360))

    string, column = find_a_place(image, latitude, longitude)

    if (string - length < 0) or (string + length >= image.shape[0]) or (column - width < 0) or (column + width >= image.shape[1]):
        return None

    return image[string - length : string + length, column - width : column + width, 0:3]

Переводим каждый цвет в единицу Добсона согласно шкале измерения

In [ ]:
DobsonUnit = {
    (156, 205, 205) : 100,
    (98, 156, 156) : 125,
    (255, 0, 255) : 150,
    (131, 98, 255) : 175,
    (98, 156, 255) : 200,
    (123, 205, 255) : 225,
    (156, 255, 255) : 250,
    (205, 255, 205) : 275,
    (0, 255, 0) : 300,
    (156, 255, 0) : 325,
    (205, 255, 98) : 350,
    (255, 255, 156) : 375,
    (255, 255, 0) : 400,
    (255, 205, 0) : 425,
    (255, 156, 98) : 450,
    (255, 98, 0) : 475,
    (255, 0, 0) : 500,
    (156, 0, 205) : 525,
    (205, 98, 205) : 550,
    (255, 156, 205) : 575,
    (255, 205, 205) : 600
}

Вычисляем среднее значение толщины озонового слоя в данной области в DU

In [ ]:
def get_dobson_unit(image):

    sum = 0;
    count = 0;
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            color = (image[i][j][0], image[i][j][1], image[i][j][2])
            if color in DobsonUnit:
                sum += DobsonUnit[color]
                count += 1
    if count > 0:
        return round(sum / count, 2)
    else:
        return None

Для каждого землетрясения рассчитываем значение толщины озонового слоя в зоне  землетрясения за 10 дней до землетрясения. Если для какого-то из дней не было найдено озоновой карты, то данное землетрясение удаляем из рассмотрения

In [ ]:
data = pd.read_csv('./earthquakes.csv')
data['-10_days'] = -1
data['-9_days'] = -1
data['-8_days'] = -1
data['-7_days'] = -1
data['-6_days'] = -1
data['-5_days'] = -1
data['-4_days'] = -1
data['-3_days'] = -1
data['-2_days'] = -1
data['-1_days'] = -1
data['0_days'] = -1

In [ ]:
for i in range(data.shape[0]):

    count_date = data.iloc[i]['time'][0:10]
    dobson_unit_list = []

    for j in range(-10, -1):
        start_date = datetime(int(count_date[0:4]), int(count_date[5:7]), int(count_date[8:10]))
        if j < 0:
            result_date = str(start_date - timedelta(days=-j))[0:10].replace('-', '')
        else:
            result_date = str(start_date + timedelta(days=j))[0:10].replace('-', '')

        # находим нужную озоновую карту
        image = None
        with ZipFile("ozone_maps.zip", "r") as myzip:
            for item in myzip.infolist():
                if item.filename[-12:-4] == result_date:
                    image = imread('ozone_maps.zip/' + item.filename)
                    image = image[58:598, 36:910, 0:3]

        if image is None:
            break

        # выделяем на ней зону землетрясения
        earthquake_zone = get_earthquake_zone(image, data.iloc[i]['latitude'], data.iloc[i]['longitude'], data.iloc[i]['mag'])

        if earthquake_zone is None:
            break

        # считаем значение толщины  озонового слоя в зоне землетрясения
        dobson_unit = get_dobson_unit(earthquake_zone)

        if dobson_unit is None:
            break

        dobson_unit_list.append(dobson_unit)

    # если для всех дней найдены озоновые карты, и получено значение толщины озонового слоя, то обновляем data
    if len(dobson_unit_list) == 11:
        for j in range(11):
            data.iloc[i, j+5] = dobson_unit_list[j]

# оставляем только нужные землетрясения
data = data[data['-10_days'] != -1]

# **Генерация точек наблюдений, в которых землетрясения не регистрировались**

In [ ]:
import random

In [ ]:
data = pd.read_csv('./earthquakes.csv')
# для поиска нужных озоновых карт меняем формат представления столбца 'time'
data['new_time'] = data['time'].apply(lambda x: str(x[0:10]).replace('-', ''))
data.drop(columns='time', inplace=True)

Проверяем, лежит ли наша точка в зоне какого-нибудь землетрясения

In [ ]:
def check_earthquake_zone(latitude, longitude, magnitude, checked_latitude, checked_longitude):

    length = 10 ** (0.43 * magnitude) / 111.1
    width = 10 ** (0.43 * magnitude) / (111.1 * math.cos(latitude * math.pi / 180))

    if latitude - length <= checked_latitude <= latitude + length and longitude - width <= checked_longitude <= longitude + width:
        return False

    return True



Создаем набор данных, в который будем добавлять сгенерированные точки

In [ ]:
number_of_points = 16000

data_without_earthquaks = pd.DataFrame(
    [{'date': 'a',
      'latitude' : 0,
      'longitude' : 0,
      '-10_days' : 0,
      '-9_days' : 0,
      '-8_days' : 0,
      '-7_days' : 0,
      '-6_days' : 0,
      '-5_days' : 0,
      '-4_days' : 0,
      '-3_days' : 0,
      '-2_days' : 0,
      '-1_days' : 0,
      '0_days' : 0,
      }] * number_of_points
    )

In [ ]:
current_number = 0
random.seed(42)

while current_number < number_of_points:

    flag = True

    year = random.randint(2015, 2023)
    month = random.randint(1, 12)
    day = random.randint(1, 28)
    date = datetime(year, month, day)
    latitude = random.randint(-90, 89) + random.random()
    longitude = random.randint(-180, 179) + random.random()

    dobson_unit_list = []

    for j in range(-10, 1):
        if j < 0:
            result_date = str(date - timedelta(days=-j))[0:10].replace('-', '')
        else:
            result_date = str(date + timedelta(days=j))[0:10].replace('-', '')

        # находим нужную озоновую карту
        image = None
        with ZipFile("ozone_maps.zip", "r") as myzip:
            for item in myzip.infolist():
                if item.filename[-12:-4] == result_date:
                    image = imread('ozone_maps.zip/' + item.filename)
                    image = image[58:598, 36:910, 0:3]

        if image is None:
            break

        # находим все землетрясения, которые произошли в эту дату
        current_data = data[data['new_time'] == result_date]

        # проверяем, чтобы сгенерированная точка не лежала ни в какой из этих зон землетрясений
        for j in range(current_data.shape[0]):
            if not check_earthquake_zone(current_data.iloc[j]['latitude'], current_data.iloc[j]['longitude'],\
                                         current_data.iloc[j]['mag'], latitude, longitude):
                flag = False
                break

        # если флаг опустился, то точка нам не подходит
        if not flag:
            break

        # выделяем область, где будем считать значение толщины озонового слоя
        not_earthquake_zone = get_earthquake_zone(image, latitude, longitude, 5.0)

        if not_earthquake_zone is None:
            break

        # считаем толщину озонового слоя
        dobson_unit = get_dobson_unit(not_earthquake_zone)

        if dobson_unit is None:
            break

        dobson_unit_list.append(dobson_unit)

    # если для всех дней условие выполняется, то добавляем взятую точку в наш датасет
    if len(dobson_unit_list) == 11:
        data_without_earthquaks.iloc[current_number, 0] = date
        data_without_earthquaks.iloc[current_number, 1] = latitude
        data_without_earthquaks.iloc[current_number, 2] = longitude

        for j in range(11):
            data_without_earthquaks.iloc[current_number, j+3] = dobson_unit_list[j]

        current_number += 1
